### Branch-and-Bound for prime programs of the form min cx s.t. Ax >= b, x is prime

In [ ]:
import numpy as np
import sympy
import gurobipy as gp
from gurobipy import GRB

In [2]:
def furtherFromPrime(x):
    '''
    Find an element of list x which has the highest distance from its closest prime number
    
    Input argument: x (list of numbers)
    
    Output: [i , j] where i is the value of that element and j is its index of the list x
    '''
    j = 0
    max_ = -float('inf')
    furthest = [-1, -1]
    for i in x:
        round_i = int(i)
        # Check if round(i) is a prime
        if sympy.isprime(round_i):
            distance = abs(i - round_i)
        else:
            next_prime = sympy.nextprime(round_i)
            prev_prime = sympy.prevprime(round_i)
            distance = min(i - prev_prime, next_prime - i)
        if distance > max_:
            max_ = distance
            furthest = [i,j]
        j += 1
    return furthest

def testInteger(x):
    '''
    Test if the given list of number are all integers
    
    Input argument: x (list of numbers)
    
    Output: Boolean, if all elements of x are integer or not
    '''
    test = np.round(x, 6) - np.round(x, 0)
    test = np.abs(test)
    tolerance = np.ones(len(test)) * -1e-6
    if all(test < tolerance) or all(test == 0):
        return True
    else:
        return False

def testPrime(x):
    '''
    Test if the given list of number are all prime numbers
    
    Input argument: x (list of numbers)
    
    Output: Boolean, if all elements of x are prime or not
    '''
    # we have to first test if it is an integer solution
    if testInteger(x):
        test = np.round(x, 6)
        s = 0
        for t in test:
            if not sympy.isprime(int(t)):
                s += 1
        if s==0:
            return True
        else:
            return False
    else:
        return False

def furtherFromInteger(x):
    '''
    Find an element of list of numbers x which is furthest from integer value
    
    Input: x (list of numbers)
    
    Output: [i, j] where i is the value of that element and j is its index 
    '''
    j = 0
    max_ = -float('inf')
    furthest = [-1, -1]
    flag = False
    for i in x:
        if abs(i-round(i, 0)) > 0.5:
            flag = True
        if flag == False and round(abs(i-round(i, 0)), 6) > round(max_, 6):
            max_ = round(abs(i-round(i, 0)), 6)
            if max_ != 0:
                furthest[0] = i
                furthest[1] = j
        elif flag == True and round((round(i, 0)+1)-i, 6) > round(max_, 6):
            max_ = round((round(i, 0)+1)-i, 6)
            if max_ != 0:
                furthest[0] = i
                furthest[1] = j
        flag = False
        j += 1
    return furthest

In [10]:
def branch_and_bound_prime(c, A, b, lower_bounds, upper_bounds):
    '''
    Implementation of branch and bound algorithm to solve prime number programming problems of the form
    
    min c x
    s.t. A_ub x <= b_ub
         A_eq = b_eq
         x in bounds
         x is prime
    
    using most fractional value branching strategy and always select the first stored node
    
    Input: 
        c list of numbers
        A_ub list of lists
        b_ub list of numbers
        A_eq list of lists
        b_eq list of numbers
        bounds list of coordinate tuple have to indicate bounds as (-inf, inf) if there is no limitation 
                and lower bounds need to manually add 2 (defaults value should be (2, inf))
    
    Output: 
        upper_z: optimal objective value
        incumbent: prime number solution to the problem
    
    '''
    n = len(c)
    # Create optimization problem
    #Suppressing Gurobi output
    env = gp.Env(empty=True)
    env.setParam("OutputFlag",0)
    env.start()
    m = gp.Model("Prime Optimization", env = env)
    
    #Variables
    v_name = {}
    x = {}
    for i in range(n):
        v_name[i] = "x{0}".format(i)
        x[i] = m.addVar(lb = lower_bounds[i], ub = upper_bounds[i], name=v_name[i])
        
    m.update()
    
    # add constraints
    for i in range(len(A)):
        m.addConstr(gp.quicksum(A[i][j]*x[j] for j in range(len(A[i]))) <= b[i])
    
    # set objective
    m.setObjective(gp.quicksum(c[i]*x[i] for i in range(len(c))), GRB.MINIMIZE)
    
    # optimize
    m.optimize()
    
    #Check if feasible
    if m.status !=2:
        print("Problem is infeasible!")
        return (0,0,0,0)
    else:
        # if the root node gives a feasible solution
        if testPrime(m.x):
            return (1, m.objval, m.x)
    
    #Intializing global upper bound and incumbent
    global_ub = 1e12
    incumbent = []
    
    # print("Initial objective value is ", m.objval)
    nodes_model = [m]
    
    #Node count
    node_count = 1
    
    #------------------------------------
    # Starting branch-and-bound iteration
    #------------------------------------ 
    
    while( len(nodes_model) != 0):
        
        #------------------------------------------------------
        #Depth first search: Last node added to the list is choosen
        #-------------------------------------------------------
        m = nodes_model[-1]
        m.optimize()
        # print(m.lb)
        # print(m.ub)
        
        # prune the node by infeasiblility or pruned by bound
        if m.status != 2:
            #Removing parent node
#             print("This node is pruned by infeasibility")
            nodes_model.pop(-1)
            continue
        elif m.objval >= global_ub:
#             print("This node is pruned by bound")
            #Removing parent node
            nodes_model.pop(-1)
            continue
        else:
#             print("Relaxation solution of the current node is ", m.x)
            # update the global upper bound
            if m.objval < global_ub and testPrime(m.x):
                global_ub = m.objval
                # print("global upper bound is updated to ", global_ub)
                incumbent = m.x
                # prune by optimality
#                 print("This node is pruned by optimality")
                nodes_model.pop(-1)
                continue
            else:
                # find which variables to be branched
                # find variable that is furthest from prime
                branch_ind = int(furtherFromPrime(m.x)[1])
                # print('Branch index = ', branch_ind)
            
                # subproblem 1 branch down
                left_model = m.copy()
                branch_val = sympy.prevprime(m.x[branch_ind])
                # print('Branch value previous prime = ', branch_val)
        
                # Generate variable
                x = {}
                for i in range(n):
                    x[i] = left_model.getVarByName(v_name[i])
            
                x[branch_ind].setAttr("UB", branch_val)
                left_model.update()
        
                # subproblem 2 branch up
                right_model = m.copy()
                branch_val = sympy.nextprime(m.x[branch_ind])
                # print('Branch value next prime = ', branch_val)
        
                # create variables
                x = {}
                for i in range(n):
                    x[i] = right_model.getVarByName(v_name[i])
            
                x[branch_ind].setAttr("LB", branch_val)
                right_model.update()
                
                #Removing parent node
                nodes_model.pop(-1)
        
                # Stored the node      
                # nodes_model.append(left_model)
                nodes_model.append(right_model)
                nodes_model.append(left_model)
        
                node_count += 2
        
    return (node_count, global_ub, incumbent)

### Input matrices 

In [11]:
c = np.array([-1, -4])
A = np.array([[3, 1], [1, 5]])
b = np.array([2000, 450])
lower_bounds=np.array([2,2])
upper_bounds = np.array([100, 100])

# c = np.array([-1,0])
# A = np.array([[-1, -1], [1, 1]])
# b = np.array([-1000, 1000])
# lower_bounds=np.array([2,2])
# upper_bounds = np.array([100000, 100000])

# c = np.array([-1, 1, 0, 0])
# A = np.array([[-1, -1, -1, -1], [1, 1, 1, 1]])
# b = np.array([-10012, 10012])
# lower_bounds=np.array([2,2,2,2])
# upper_bounds = np.array([10000000,10000000,10000000,10000000])

# c = np.array([-10, -64, -118, -142])
# A = np.array([[3,5,10,14],[3,8,6,12],[3,7,11,14]])
# b = np.array([800,800,900])
# lower_bounds=np.array([2,2,2,2])
# upper_bounds = np.array([80, 400, 900, 94])

# c = np.array([-1, -1, -2])
# A = np.array([[3,5,10],[3,8,1], [3,2,1]])
# b = np.array([10000,10000,10000])
# lower_bounds=np.array([2,2,2])
# upper_bounds = np.array([997, 997, 997])

In [12]:
node_count, global_ub, incumbent = branch_and_bound_prime(c, A, b, lower_bounds, upper_bounds)
print(node_count, global_ub, incumbent)

13 -375.0 [83.0, 73.0]
